In [ ]:
import sys
from pathlib import Path

import torch
from torch import GradScaler, nn, optim
from torch.utils import data
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.models import segmentation

sys.path.append(str(Path("..").resolve()))
from src.datasets import DATASET_ZOO
from src.models import MODEL_ZOO
from src.pipeline import LocalLogger, Trainer, init_logging
from src.utils.rng import seed
from src.utils.transform import DataAugment, DataTransform

In [ ]:
root = Path(r"D:\_Dataset")
entry = DATASET_ZOO["VOC"]
train_dataset = entry.train_constructor(
    root=root, transforms=DataTransform((500, 500)), year="2012"
)
# train_dataset = data.Subset(train_dataset, range(50))
val_dataset = entry.val_constructor(root=root, transforms=DataTransform(), year="2012")
# val_dataset = data.Subset(val_dataset, range(50))
train_loader = DataLoader(train_dataset, batch_size=2, drop_last=True)
val_loader = DataLoader(val_dataset)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = MODEL_ZOO["deeplabv3_resnet50"](num_classes=entry.num_classes, aux_loss=True)
criterion = nn.CrossEntropyLoss(ignore_index=entry.ignore_index)
optimizer = optim.SGD(model.parameters(), lr=3e-4, momentum=0.9, weight_decay=5e-4)

run_folder = Path("../runs/exp1")

In [ ]:
trainer = Trainer(
    model,
    train_loader,
    DataAugment(hflip=0.5),
    val_loader,
    DataAugment(),
    criterion,
    optimizer,
    optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5),
    GradScaler(device),
    device,
    4,
    10,
    {"aux": 0.5},
    entry.num_classes,
    entry.labels,
    entry.colors,
    run_folder,
    checkpoint_epochs=5,
    loggers=(LocalLogger(run_folder, entry.labels),),
)

In [ ]:
seed(42)
run_folder.mkdir(parents=True)
with init_logging(run_folder / "run.log"):
    trainer.train()